# Predictions

In [1]:
import os

import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from tqdm.notebook import tqdm

In [2]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
sample_submission_df = pd.read_csv(
    "../../data/amex-default-prediction/sample_submission.csv"
).set_index("customer_ID")
sample_submission_df

,prediction
customer_ID,
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0
00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397d4263dafa1daedef5,0
0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5e400fc98e7bd43ce8,0
00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf6e56734528702d694,0
00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9a4693dd914fca22557,0
...,...
ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c5d60460dba6dedc41e,0
ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3a4f0ca3de613b0b2ad,0
ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475cb095d2443a68030f1,0


<IPython.core.display.Javascript object>

In [4]:
DATA_DIR = "../../data/amex-default-prediction/test"

<IPython.core.display.Javascript object>

In [5]:
!ls $DATA_DIR

test_group_data_10.csv	test_group_data_1.csv	test_group_data_29.csv
test_group_data_11.csv	test_group_data_20.csv	test_group_data_2.csv
test_group_data_12.csv	test_group_data_21.csv	test_group_data_30.csv
test_group_data_13.csv	test_group_data_22.csv	test_group_data_3.csv
test_group_data_14.csv	test_group_data_23.csv	test_group_data_4.csv
test_group_data_15.csv	test_group_data_24.csv	test_group_data_5.csv
test_group_data_16.csv	test_group_data_25.csv	test_group_data_6.csv
test_group_data_17.csv	test_group_data_26.csv	test_group_data_7.csv
test_group_data_18.csv	test_group_data_27.csv	test_group_data_8.csv
test_group_data_19.csv	test_group_data_28.csv	test_group_data_9.csv


<IPython.core.display.Javascript object>

In [6]:
def do_work(step, file_name, model):
    X_test = pd.read_csv(os.path.join(DATA_DIR, file_name)).set_index("customer_ID")

    return pd.DataFrame(
        {
            "customer_ID": X_test.index,
            "prediction": model.predict_proba(X_test)[:, 1],
        }
    ).set_index("customer_ID")

<IPython.core.display.Javascript object>

In [7]:
model = CatBoostClassifier()

model_path = os.path.join(os.path.dirname(DATA_DIR), "models", "catboost-model_v1.cbm")
model.load_model(model_path)

preds = []

for step, file_name in enumerate(tqdm(os.listdir(DATA_DIR)), 1):
    sub_preds = do_work(step, file_name, model)
    preds.append(sub_preds)

  0%|          | 0/30 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

In [8]:
submission = pd.concat(preds).sort_values("customer_ID")
submission

,prediction
customer_ID,
00000469ba478561f23a92a868bd366de6f6527a684c9a2e78fb826dcac3b9b7,0.033925
00001bf2e77ff879fab36aa4fac689b9ba411dae63ae397d4263dafa1daedef5,0.004444
0000210045da4f81e5f122c6bde5c2a617d03eef67f82c5e400fc98e7bd43ce8,0.027220
00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976cf6e56734528702d694,0.520299
00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9a4693dd914fca22557,0.721996
...,...
ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c5d60460dba6dedc41e,0.007883
ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3a4f0ca3de613b0b2ad,0.659939
ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475cb095d2443a68030f1,0.693675


<IPython.core.display.Javascript object>

In [9]:
save_path = os.path.join(os.path.dirname(DATA_DIR), "submission_v1.csv.gz")
submission.to_csv(save_path, compression="gzip")

<IPython.core.display.Javascript object>